<a target="_blank" href="https://colab.research.google.com/github/Sebelino/DD2424-project/blob/main/e1_sebastian.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%load_ext autoreload
%autoreload 2

import os
import socket

repo_name = "DD2424-project"
dep_filename = "download_dataset.py"
cwd = os.getcwd()
hostname = socket.gethostname()

print(f"Host: {hostname}")
print(f"You are here: {cwd}")

Host: deeplearning-vm
You are here: /home/sebelino7_gmail_com/pycharm_depl


In [2]:
from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import
print("Training results should now be reproducible.")


Training results should now be reproducible.


# Train model

In [3]:
from typing import Literal
import augmentation
from augmentation import AugmentationParams
import json
from training import TrainParams, NagParams
from datasets import DatasetParams
from search import run_search, SearchParams

dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    trainval_size=None,
    #trainval_size=400,
    validation_set_fraction=0.2,
)

architecture: Literal["resnet18", "resnet34", "resnet50"] = "resnet50"
training_params = TrainParams(
    seed=determinism.seed,
    architecture=architecture,
    n_epochs=20,
    optimizer=NagParams(
        learning_rate=None,  # Included in search
        weight_decay=None,  # Included in search
        momentum=None,  # Included in search
    ),
    freeze_layers=True,
    unfreezing_epochs=(None, None),  # Included in search
    augmentation=AugmentationParams(
        enabled=None, # Included in search
        transform=augmentation.to_transform(architecture, ["resize", "flip"])
    ),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
)

search_params = SearchParams(
    learning_rate=lambda trial: trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
    weight_decay=lambda trial: trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True),
    momentum=lambda trial: trial.suggest_float("momentum", 0.7, 0.99, log=False),
    augmentation=lambda trial: trial.suggest_categorical("augmentation", [False,True]),
    unfreeze_epoch_1=lambda trial: trial.suggest_int("unfreeze_epoch_1", 1, 5),
    unfreeze_epoch_2=lambda trial, unfreeze_epoch_1: trial.suggest_int(
        "unfreeze_epoch_2",
        low=unfreeze_epoch_1 + 1,
        high=9
    ),
)

search_result = run_search(dataset_params, training_params, search_params, determinism, n_trials=219)

print("Search result:")
print(json.dumps(search_result, indent=2))

[I 2025-05-12 19:50:07,320] A new study created in memory with name: no-name-7fac3da6-5b48-4f44-872d-edd5c7a8bbb5


Running study with dataset parameters:
{
  "splitting_seed": 42,
  "shuffler_seed": 42,
  "batch_size": 32,
  "validation_set_fraction": 0.2,
  "target_types": "category",
  "labelled_data_fraction": 1.0,
  "class_fractions": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}
and training parameters:
{
  "seed": 42,
  "architecture": "resnet50",
  "optimizer": {
    "name": "nag"
  },
  "n_epochs": 20,
  "freeze_layers": true,
  "unfreezing_epochs": [],
  "validation_freq": 1,
  "augmentation": {
    "transform": "Compose(\n    RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)\n    RandomHorizontalFlip(p=0.5)\n    ToTensor()\n    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n)"
  },
  "unsup_weight": 0.5,
  "masked_finetune": false,
  "mask_K": 1,
  "contrastive_temp"